### set parameters

In [8]:
conf_name = 'standard'
output_name = 'yolo_n_weighted_sampler_augment_tuned'
columns_suffix = '_n_weighted_tuned'
output_folder = r"C:\Users\scanman\Documents\Jannik_Vikari\final_results\python_scripts\compare_pred_gt_mobile_phone"

yolo_model = r"C:\Users\scanman\Documents\Jannik_Vikari\final_results\python_scripts\tree_species_weighted_sampler\train_yolo_weighted_sampler_augment_tuned\runs\segment\train\weights\best.pt"
img_dataset = r"C:\Users\scanman\Documents\Jannik_Vikari\final_results\image_data\tree_species_data_15_01_24\test\images"

# create folder if it does not exist
from pathlib import Path
Path(output_folder).mkdir(parents=True, exist_ok=True)

### load model and do validation

In [9]:
from ultralytics import YOLO

model = YOLO(yolo_model)

metrics = model.predict(img_dataset,
                    project = output_folder,
                    name = f'pred_{output_name}_conf_{conf_name}',
                    save_txt=True)


image 1/870 C:\Users\scanman\Documents\Jannik_Vikari\final_results\image_data\tree_species_data_15_01_24\test\images\1667566182238_jpg.rf.52f94cfd698c80eb004814db5ee324f4.jpg: 640x480 1 betula_spp, 99.6ms
image 2/870 C:\Users\scanman\Documents\Jannik_Vikari\final_results\image_data\tree_species_data_15_01_24\test\images\1667979828166_jpg.rf.552e942abb6f8a8734ff185e68e79005.jpg: 640x480 1 picea_abies, 7.7ms
image 3/870 C:\Users\scanman\Documents\Jannik_Vikari\final_results\image_data\tree_species_data_15_01_24\test\images\1667979852029_jpg.rf.3ab6735911e6bfbb6e60dac2f058da06.jpg: 640x480 1 picea_abies, 13.1ms
image 4/870 C:\Users\scanman\Documents\Jannik_Vikari\final_results\image_data\tree_species_data_15_01_24\test\images\1667981532427_jpg.rf.f00778e424f5c98e566be9083d8046ea.jpg: 640x480 1 picea_abies, 16.7ms
image 5/870 C:\Users\scanman\Documents\Jannik_Vikari\final_results\image_data\tree_species_data_15_01_24\test\images\1667981601584_jpg.rf.841d11f3ec69c694bf846f77ca842e1d.jpg: 6

### extract necessary validation metrics for all classes and each class seperatly

In [5]:
# copy the confusion matrix to desired output folder and rename
import os
import shutil

# get conf and iou of confusion matrix
confidence_mat = metrics.confusion_matrix.conf
iou_mat = metrics.confusion_matrix.iou_thres

# get files of confusion matrix
confusion_mat = os.path.join(output_folder, f'val_{output_name}_conf_{conf_name}', 'confusion_matrix.png')
confusion_mat_normal = os.path.join(output_folder, f'val_{output_name}_conf_{conf_name}', 'confusion_matrix_normalized.png')

# Specify destination (including the new file name)
dest_conf_mat = os.path.join(output_folder, f"confusion_matrix_conf_{confidence_mat}_iou_{iou_mat}.png")
dest_conf_mat_normal = os.path.join(output_folder, f"confusion_matrix_normalized_conf_{confidence_mat}_iou_{iou_mat}.png")

# Copy the file
shutil.copy2(confusion_mat, dest_conf_mat)
shutil.copy2(confusion_mat_normal, dest_conf_mat_normal)

'C:\\Users\\scanman\\Documents\\Jannik_Vikari\\final_results\\python_scripts\\final_python_code_rep\\master-thesis\\results\\compare_Species_Seg_mobil\\confusion_matrix_normalized_conf_0.25_iou_0.45.png'

##### metrics for per class

In [11]:
# get class indexes
class_index = metrics.ap_class_index

# box precisions for all classes
p_box = metrics.box.p

# box recall for all classes
r_box = metrics.box.r

# box f1 for all classes
f1_box = metrics.box.f1

# box average precision for all classes, iou thresholds between 0.5 and 0.95: ap50-95
ap_box = metrics.box.ap

# box average precision for all classes, iou threshold 0.5: ap50
ap50_box = metrics.box.ap50

# mask precisions for all classes
p_mask = metrics.seg.p

# mask recall for all classes
r_mask = metrics.seg.r

# mask f1 for all classes
f1_mask = metrics.seg.f1

# mask average precision for all classes, iou thresholds between 0.5 and 0.95: ap50-95
ap_mask = metrics.seg.ap

# mask average precision for all classes, iou threshold 0.5: ap50
ap50_mask = metrics.seg.ap50

##### mean metrics for all classes

In [12]:
# box mean precision
mp_box = metrics.box.mp

# box mean recall 
mr_box = metrics.box.mr

# box mean average precision for all classes, iou thresholds between 0.5 and 0.95: ap50-95
map_box = metrics.box.map

# box mean average precision for all classes, iou threshold 0.5: ap50
map50_box = metrics.box.map50


# mask mean precision
mp_mask = metrics.seg.mp

# mask mean recall 
mr_mask = metrics.seg.mr

# mask mean average precision for all classes, iou thresholds between 0.5 and 0.95: ap50-95
map_mask = metrics.seg.map

# mask mean average precision for all classes, iou threshold 0.5: ap50
map50_mask = metrics.seg.map50

#### combine to one dataframe

In [13]:
import os
import pandas as pd
import numpy as np

dataset = pd.DataFrame({'class_index': class_index,
                        f'precision_box{columns_suffix}': p_box,
                        f'recall_box{columns_suffix}': r_box,
                        f'map_50_box{columns_suffix}': ap50_box,
                        f'map_50_95_box{columns_suffix}': ap_box,
                        f'f1_box{columns_suffix}': f1_box,
                        f'precision_mask{columns_suffix}': p_mask,
                        f'recall_mask{columns_suffix}': r_mask,
                        f'map_50_mask{columns_suffix}': ap50_mask,
                        f'map_50_95_mask{columns_suffix}': ap_mask,
                        f'f1_mask{columns_suffix}': f1_mask})


dataset_mean = pd.DataFrame({'class_index': 'all',
                             f'precision_box{columns_suffix}': mp_box,
                             f'recall_box{columns_suffix}': mr_box,
                             f'map_50_box{columns_suffix}': map50_box,
                             f'map_50_95_box{columns_suffix}': map_box,
                             f'f1_box{columns_suffix}': np.mean(f1_box),
                             f'precision_mask{columns_suffix}': mp_mask,
                             f'recall_mask{columns_suffix}': mr_mask,
                             f'map_50_mask{columns_suffix}': map50_mask,
                             f'map_50_95_mask{columns_suffix}': map_mask,
                             f'f1_mask{columns_suffix}': np.mean(f1_mask)}, index=[0])

final_dataset = pd.concat([dataset_mean, dataset], ignore_index=True)

# species data frame
species_classes = pd.DataFrame(list(model.names.items()), columns=['class_index', 'species'])

species_all = pd.DataFrame({'class_index': 'all', 'species': 'all'}, index=[0])

species_df = pd.concat([ species_all, species_classes], ignore_index=True)

# combine final df and species df
out_df = species_df.merge(final_dataset, on="class_index")

filename = f'metrics_{output_name}_conf_{conf_name}.txt'

output_file_path = os.path.join(output_folder, filename)

out_df.to_csv(output_file_path, sep='\t', index=False)